In [60]:
import backtrader as bt
import yfinance as yf
import pandas as pd
import datetime

In [61]:
# 下載股票資料
df = yf.download('AAPL', start='2020-01-01', end='2023-01-01')

# 清理掉多重索引
df.columns = df.columns.droplevel(1)
df

/var/folders/k0/mjnvdkdj7sn_zbtz76h68xr40000gn/T/ipykernel_68069/534702881.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('AAPL', start='2020-01-01', end='2023-01-01')
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Date,,,,,
2020-01-02,72.468285,72.528604,71.223282,71.476622,135480400
2020-01-03,71.763733,72.523762,71.539345,71.696175,146322800
2020-01-06,72.335556,72.374162,70.634539,70.885472,118387200
2020-01-07,71.995369,72.600975,71.775804,72.345220,108872000
2020-01-08,73.153503,73.455103,71.698589,71.698589,132079200
...,...,...,...,...,...
2022-12-23,129.900269,130.451943,127.713261,128.974237,63814900
2022-12-27,128.097473,129.456968,126.806945,129.427415,69007800
2022-12-28,124.166786,129.082622,123.999315,127.742834,85438400


In [62]:
# 改成 bt 可接受的格式
data = bt.feeds.PandasData(dataname=df)

In [63]:
# 設定回測執行的大腦
cerebro = bt.Cerebro()

# 將資料加到回測大腦中
cerebro.adddata(data)

# 設定初始資金
cerebro.broker.setcash(100000)

# 設定交易手續費
cerebro.broker.setcommission(commission=0.001) 

In [64]:
import backtrader as bt

class MyMaCrossStrategy(bt.Strategy):
    params = (
        ('fast_period', 10),
        ('slow_period', 30),
    )

    def __init__(self):
        # 收盤價
        self.dataclose = self.datas[0].close
        
        # 移動平均線
        self.fast_ma = bt.indicators.SMA(self.dataclose, period=self.p.fast_period)
        self.slow_ma = bt.indicators.SMA(self.dataclose, period=self.p.slow_period)

        # 上一根是否為金叉，方便偵測交叉
        self.crossover = bt.indicators.CrossOver(self.fast_ma, self.slow_ma)

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} - {txt}')
    
    def next(self):
        # 印出 debug 資訊
        self.log(f"Fast: {self.fast_ma[0]:.2f}, Slow: {self.slow_ma[0]:.2f}")

        # 金叉（快線上穿慢線）
        if self.crossover > 0:
            if not self.position:  # 沒持倉才買
                self.buy()
                self.log(f"BUY EXECUTED: Close={self.dataclose[0]:.2f}")

        # 死叉（快線下穿慢線）
        elif self.crossover < 0:
            if self.position:  # 有持倉才賣
                self.sell()
                self.log(f"SELL EXECUTED: Close={self.dataclose[0]:.2f}")


In [65]:
# 執行策略
print("\nRunning backtest...")

print(f"Starting Portfolio Value: {cerebro.broker.getvalue():,.2f}")

# 加入策略
cerebro.addstrategy(MyMaCrossStrategy, fast_period=20, slow_period=60)
# Run the backtest
results = cerebro.run()

# Get the strategy instance
first_strategy = results[0]

print(f"Final Portfolio Value: {cerebro.broker.getvalue():,.2f}")
print("Backtest complete.")


Running backtest...
Starting Portfolio Value: 100,000.00
2020-03-30 - Fast: 63.13, Slow: 71.21
2020-03-31 - Fast: 62.71, Slow: 71.04
2020-04-01 - Fast: 61.96, Slow: 70.80
2020-04-02 - Fast: 61.38, Slow: 70.59
2020-04-03 - Fast: 60.80, Slow: 70.34
2020-04-06 - Fast: 60.76, Slow: 70.16
2020-04-07 - Fast: 60.45, Slow: 69.95
2020-04-08 - Fast: 60.33, Slow: 69.75
2020-04-09 - Fast: 60.57, Slow: 69.58
2020-04-13 - Fast: 60.51, Slow: 69.42
2020-04-14 - Fast: 61.06, Slow: 69.31
2020-04-15 - Fast: 61.44, Slow: 69.18
2020-04-16 - Fast: 61.92, Slow: 69.06
2020-04-17 - Fast: 62.38, Slow: 68.92
2020-04-20 - Fast: 62.96, Slow: 68.76
2020-04-21 - Fast: 63.49, Slow: 68.56
2020-04-22 - Fast: 63.84, Slow: 68.43
2020-04-23 - Fast: 64.20, Slow: 68.26
2020-04-24 - Fast: 64.50, Slow: 68.10
2020-04-27 - Fast: 64.93, Slow: 67.93
2020-04-28 - Fast: 65.21, Slow: 67.81
2020-04-29 - Fast: 65.62, Slow: 67.73
2020-04-30 - Fast: 66.26, Slow: 67.63
2020-05-01 - Fast: 66.79, Slow: 67.51
2020-05-04 - Fast: 67.42, Slow

In [66]:
# 透過 matplotlib 繪製圖表
import matplotlib.pyplot as plt

print("\nGenerating plot...")

try:
    cerebro.plot(
        style='candlestick',
        barup='red',
        bardown='green',
        volume=True,
        iplot=False,   # 背後使用 matplotlib，而非 Bokeh
        show=False
    )
    print("Plot displayed.")

except Exception as e:
    print(f"Could not plot results: {e}")
    print("Make sure matplotlib is installed and working correctly.")


Generating plot...
Plot displayed.
